In [16]:
!pip install nltk
!pip install ipynb

In [18]:
import nltk
from nltk.stem import PorterStemmer, LancasterStemmer, WordNetLemmatizer, SnowballStemmer
from nltk.corpus import stopwords 
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/qijunjin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/qijunjin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [189]:
import pandas as pd
import scipy
import sklearn
from sklearn import *
import numpy as np
import os


from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import roc_auc_score



ImportError: cannot import name 'TfIdfVectorizer' from 'sklearn.feature_extraction.text' (/Users/qijunjin/miniforge3/envs/DESKTOP_MASTER/lib/python3.9/site-packages/sklearn/feature_extraction/text.py)

In [20]:
from ipynb.fs.full.SCV import *

### Data

In [21]:
path_data = os.path.expanduser('~') 

# use this to train and VALIDATE your solution
data = pd.read_csv("./quora_train_data.csv")

# use this to provide the expected generalization results
test_df = pd.read_csv("./quora_test_data.csv")

train_df, val_df = sklearn.model_selection.train_test_split(data, test_size=0.2,random_state=123)

print('train_df.shape=',train_df.shape)
print('val_df.shape=',val_df.shape)
print('test_df.shape=',test_df.shape)

data

train_df.shape= (258745, 6)
val_df.shape= (64687, 6)
test_df.shape= (80858, 6)


id    qid1    qid2  \
0       346692   38482   10706   
1       327668  454117  345117   
2       272993  391373  391374   
3        54070   82673   95496   
4        46450   38384   72436   
...        ...     ...     ...   
323427  192476  292119  292120   
323428   17730   33641   33642   
323429   28030   52012   52013   
323430  277869  397054  120852   
323431  249342  362958  362959   

                                                question1  \
0              Why do I get easily bored with everything?   
1       How do I study for Honeywell company recruitment?   
2           Which search engine algorithm is Quora using?   
3                           How can I smartly cut myself?   
4        How do I see who is viewing my Instagram videos?   
...                                                   ...   
323427  Is it okay to use a laptop while it is chargin...   
323428            How can dogs understand human language?   
323429                      What's your favourite lotion?   
323430          How does one become a hedge fund manager?   
323431  How did the US acquire over 80 trillion financ...   

                                                question2  is_duplicate  
0       Why do I get bored with things so quickly and ...             1  
1       How do I study for Honeywell company recruitme...             1  
2          Why is Quora not using reliable search engine?             0  
3       Can someone who thinks about suicide for 7 yea...             0  
4            Can one tell who viewed my Instagram videos?             1  
...                                                   ...           ...  
323427         Is it OK to use your phone while charging?             0  
323428            Can dogs understand the human language?             0  
323429                 What's your favourite skin lotion?             1  
323430   What should I do to become a hedge fund manager?             1  
323431  We've thought about evil geniuses ruling the w...             0  

[323432 rows x 6 columns]

### Functions

In [22]:
def cast_list_as_strings(mylist):
    """
    return a list of strings
    """
    assert isinstance(mylist, list), f"the input mylist should be a list it is {type(mylist)}"
    
    mylist_of_strings = []
    for x in mylist:
        mylist_of_strings.append(str(x))

    return mylist_of_strings


def get_features_from_df(df, count_vectorizer):
    """
    returns a sparse matrix containing the features build by the count vectorizer.
    Each row should contain features from question1 and question2.
    """
    q1_casted =  cast_list_as_strings(list(df["question1"]))
    q2_casted =  cast_list_as_strings(list(df["question2"]))
    
    ############### Begin exercise ###################
    # what is kaggle                  q1
    # What is the kaggle platform     q2
    X_q1 = count_vectorizer.transform(q1_casted)
    X_q2 = count_vectorizer.transform(q2_casted)    
    X_q1q2 = scipy.sparse.hstack((X_q1,X_q2))
    ############### End exercise ###################

    return X_q1q2


def get_mistakes(clf, X_q1q2, y):

    ############### Begin exercise ###################
    predictions = clf.predict(X_q1q2)
    incorrect_predictions = predictions != y 
    incorrect_indices, = np.where(incorrect_predictions)
    
    ############### End exercise ###################
    
    if np.sum(incorrect_predictions)==0:
        print("no mistakes in this df")
    else:
        return incorrect_indices, predictions
    
    
def print_mistake_k(k, mistake_indices, predictions):
    print(train_df.iloc[mistake_indices[k]].question1)
    print(train_df.iloc[mistake_indices[k]].question2)
    print("true class:", train_df.iloc[mistake_indices[k]].is_duplicate)
    print("prediction:", predictions[mistake_indices[k]])

### Preprocess

In [23]:
q1_train =  cast_list_as_strings(list(train_df["question1"]))
q2_train =  cast_list_as_strings(list(train_df["question2"]))
q1_val  =  cast_list_as_strings(list(val_df["question1"]))
q2_val  =  cast_list_as_strings(list(val_df["question2"]))
q1_test  =  cast_list_as_strings(list(test_df["question1"]))
q2_test  =  cast_list_as_strings(list(test_df["question2"]))

### Feature

In [24]:
all_questions = q1_train + q2_train

In [25]:
#count_vectorizer = CountVectorizer(ngram_range=(1,2))

In [26]:
def int_to_en(num):
    d = { 0 : 'zero', 1 : 'one', 2 : 'two', 3 : 'three', 4 : 'four', 5 : 'five',
          6 : 'six', 7 : 'seven', 8 : 'eight', 9 : 'nine', 10 : 'ten',
          11 : 'eleven', 12 : 'twelve', 13 : 'thirteen', 14 : 'fourteen',
          15 : 'fifteen', 16 : 'sixteen', 17 : 'seventeen', 18 : 'eighteen',
          19 : 'nineteen', 20 : 'twenty',
          30 : 'thirty', 40 : 'forty', 50 : 'fifty', 60 : 'sixty',
          70 : 'seventy', 80 : 'eighty', 90 : 'ninety' }
    k = 1000
    m = k * 1000
    b = m * 1000
    t = b * 1000

    assert(0 <= num)

    if (num < 20):
        return d[num]

    if (num < 100):
        if num % 10 == 0: return d[num]
        else: return d[num // 10 * 10] + '-' + d[num % 10]

    if (num < k):
        if num % 100 == 0: return d[num // 100] + ' hundred'
        else: return d[num // 100] + ' hundred and ' + int_to_en(num % 100)

    if (num < m):
        if num % k == 0: return int_to_en(num // k) + ' thousand'
        else: return int_to_en(num // k) + ' thousand, ' + int_to_en(num % k)

    if (num < b):
        if (num % m) == 0: return int_to_en(num // m) + ' million'
        else: return int_to_en(num // m) + ' million, ' + int_to_en(num % m)

    if (num < t):
        if (num % b) == 0: return int_to_en(num // b) + ' billion'
        else: return int_to_en(num // b) + ' billion, ' + int_to_en(num % b)

    if (num % t == 0): return int_to_en(num // t) + ' trillion'
    else: return int_to_en(num // t) + ' trillion, ' + int_to_en(num % t)

    raise AssertionError('num is too large: %s' % str(num))

In [87]:
stpw = set(stopwords.words("english"))
stpw.remove('not')
stpw.remove('nor')
stpw.remove('no')

In [175]:
def num_conv(s):
    try:
        return int_to_en(int(s)).replace(",","").replace(" ","_")
    except:
        return s


def my_doc_cleaner(doc, pattern=r"[^a-zA-Z0-9']"):
    # Allow alphanumeric characters
    doc_cleaner_pattern = pattern
    clean_doc_pattern = re.compile(doc_cleaner_pattern)
    doc_clean = clean_doc_pattern.sub(" ", doc)

    return doc_clean.lower()


# stpw = set(stopwords.words("english"))
stpw = set(stopwords.words("english"))
stpw.remove('not')
stpw.remove('nor')
stpw.remove('no')


#stemmer =  SnowballStemmer(language='english')
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

def my_tokenizer_func(doc, 
                      ngrams=(1,3), 
                      numbers_to_words=True,
                      stop_words=None,
                      pattern=r"(?u)\b\w\S*\w*\b",
                      lem=True,
                      stem=True):
    
    # Split using a pattern
#     pat=r"(?u)\b\w\w+\b"
#     pat=r"(?u)\b\w\S*\w*\b"
    token_pattern = re.compile(pattern)
    lst = token_pattern.findall(doc)
    
    # Transform numbers into words
    #if numbers_to_words:
        #lst = list(map(lambda x: num_conv(x), lst))
        
    # Drop stopwords 
    #lst = list(filter(lambda x : x not in stop_words, lst))
    
    #Lemmatizer 
    if lem:
        lst = list(map(lambda x: lemmatizer.lemmatize(x), lst))
        
    #Stemmer
    if stem:
        lst = list(map(lambda x: stemmer.stem(x), lst))
    
    if ngrams==(1,1):
        return lst
    
    # Generate ngrams
    lstRet = []
    for a in range(ngrams[0], ngrams[1]+1):
        if a!=1:
            lstRet+=list(zip(*[lst[i:] for i in range(a)]))
    return lstRet if ngrams[0]!=1 else lst+lstRet

In [176]:
# I want to convert (don't -> do not), maintaining the negative sense
# TODO
st = set(stopwords.words("english"))
st.remove('not')
st.remove('nor')
st.remove('no')

st
lst= ['asdas','as','sd', 'not', 'on']

In [177]:
list(filter(lambda x : x not in st, lst))

['asdas', 'sd', 'not']

In [178]:
count_vectorizer = SimpleCountVectorizer(
    #stop_words = , # Don't use, have better auc_score
    doc_cleaner_func=my_doc_cleaner,
    tokenizer_func=my_tokenizer_func)

In [179]:
count_vectorizer.fit(all_questions)

  0%|          | 0/517490 [00:00<?, ?it/s]

SimpleCountVectorizer(doc_cleaner_func=<function my_doc_cleaner at 0x2a66b03a0>,
                      tokenizer_func=<function my_tokenizer_func at 0x2b353be50>)

In [180]:
X_tr_q1q2 = get_features_from_df(train_df, count_vectorizer)
X_va_q1q2 = get_features_from_df(val_df, count_vectorizer)

X_te_q1q2  = get_features_from_df(test_df, count_vectorizer)

X_tr_q1q2.shape, train_df.shape, val_df.shape, X_va_q1q2.shape, test_df.shape, X_te_q1q2.shape

((258745, 5255370),
 (258745, 6),
 (64687, 6),
 (64687, 5255370),
 (80858, 6),
 (80858, 5255370))

### Model

In [181]:
y_train = train_df["is_duplicate"].values
y_val = val_df["is_duplicate"].values
y_test = test_df['is_duplicate'].values

In [182]:
logistic = sklearn.linear_model.LogisticRegression(solver="liblinear", random_state=123)
logistic.fit(X_tr_q1q2, y_train)

LogisticRegression(random_state=123, solver='liblinear')

In [183]:
y_tr_pred = logistic.predict(X_tr_q1q2)
y_va_pred = logistic.predict(X_va_q1q2)
y_te_pred = logistic.predict(X_te_q1q2)

In [184]:
print(logistic.score(X_tr_q1q2, y_train))
print(logistic.score(X_va_q1q2, y_val))
print(logistic.score(X_te_q1q2, y_test))

0.9970820692187289
0.8075502032865954
0.802629300749462


In [185]:
print(roc_auc_score(y_train, y_tr_pred))
print(roc_auc_score(y_val, y_va_pred))
print(roc_auc_score(y_test, y_te_pred))

0.9970645815831869
0.7818433462461073
0.7764974192729127


In [186]:
mistake_indices, predictions = get_mistakes(logistic,
                                            X_tr_q1q2,
                                            y_train)

In [187]:
len(mistake_indices)

755

In [101]:
len(y_train)

258745

In [188]:
from TfIdfVectorizer import TfIdfVectorizer as tf

ModuleNotFoundError: No module named 'TfIdfVectorizer'